Valid tickers for this code, ['AMC', 'AMD', 'BBBY', 'F', 'GME', 'NVDA', 'PLTR', 'PROG', 'TSLA', 'X']

The predictions from this notebook only make sense if it is run before 4pm EST each day, since otherwise it will attempt to predict a known value

In [122]:
import urllib.request
ticker='X' 
url = 'http://cs301.pintoenterprisetech.com/'+ticker+'.csv'
urllib.request.urlretrieve(url, ticker+".csv")
url = 'http://cs301.pintoenterprisetech.com/'+ticker+'historical.csv'
urllib.request.urlretrieve(url, ticker+"historical.csv")

('Xhistorical.csv', <http.client.HTTPMessage at 0x7f6121444190>)

In [123]:
import pandas as pd
import numpy as np
import datetime

In [124]:
first_data_collection_date = datetime.datetime.strptime('2021-10-31 04:00:00', "%Y-%m-%d %H:%M:%S")

In [125]:
df = pd.read_csv(ticker+".csv")

In [126]:
daterange = pd.date_range(first_data_collection_date, datetime.datetime.today(), freq="10T")

In [127]:
temp1df = df.drop(['ticker'], axis=1)
temp1df['timestamp'] = pd.to_datetime(temp1df['timestamp'])
temp1df.set_index("timestamp", inplace=True)
temp1df = temp1df.reindex(daterange, fill_value=0, method='nearest')

In [128]:
date = first_data_collection_date
#Get the sentiment values from the same day, before 4pm into the same row of a dataframe
temp1df.loc[date+datetime.timedelta(days=0):date+datetime.timedelta(days=1)].reset_index().drop('index', axis=1).T.reset_index().drop('index', axis=1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144
0,0.9826,0.9826,-0.9285,-0.9285,0.7543,0.7543,0.0,0.0,0.7318,-0.7964,-0.7964,0.9661,0.9661,0.9661,0.9661,0.4807,0.4807,0.9705,0.9705,0.6597,0.6597,0.6597,0.9675,0.885,0.885,0.0,0.0,0.3708,0.9804,0.9804,0.9804,0.902,0.902,0.902,0.9785,0.9785,0.832125,0.832125,0.832125,0.9706,...,-0.096333,0.003667,0.003667,-0.6177,-0.6767,-0.6767,-0.6767,0.9806,0.9806,0.91528,0.94075,0.94075,-0.0516,-0.0516,-0.7581,0.63265,0.63265,-0.061125,-0.061125,0.18375,0.43885,0.43885,0.88935,0.9836,0.9836,0.9836,0.1299,0.1299,0.3612,0.3612,0.20655,0.20655,0.9451,0.19685,0.9694,0.9694,-0.09,0.9606,0.5242,0.153475


In [129]:
outputdf = pd.DataFrame()
finished = False
date = first_data_collection_date #The day our data collection starts
fillValue=0
while not finished:
  tempdf = temp1df.loc[date:date+datetime.timedelta(days=1)].reset_index().drop('index', axis=1)
  tempdf = tempdf.T.reset_index().drop('index', axis=1)
  if tempdf.shape[1] < 145:
    print(tempdf.shape)
    fillValue = tempdf.T.max().values[0]
    finished=True
  outputdf = outputdf.append(tempdf)
  date = date+datetime.timedelta(days=1)
  
outputdf.reset_index(inplace=True)
outputdf.drop('index', axis=1, inplace=True)
outputdf.fillna(method='ffill', inplace=True) #ffill will fill missing values using the previous day's values, which seems to give the best results
outputdf

(1, 66)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144
0,0.982600,0.982600,-0.9285,-0.928500,0.754300,0.754300,0.00000,0.00000,0.731800,-0.79640,-0.79640,0.966100,0.966100,0.96610,0.966100,0.480700,0.4807,0.97050,0.970500,0.659700,0.659700,0.659700,0.967500,0.885000,0.88500,0.00000,0.00000,0.37080,0.98040,0.98040,0.98040,0.90200,0.90200,0.902000,0.978500,0.978500,0.832125,0.832125,0.832125,0.97060,...,-0.096333,0.003667,0.003667,-0.617700,-0.676700,-0.676700,-0.676700,0.980600,0.980600,0.915280,0.940750,0.940750,-0.051600,-0.051600,-0.758100,0.632650,0.632650,-0.061125,-0.061125,0.183750,0.438850,0.438850,0.889350,0.983600,0.983600,0.983600,0.129900,0.129900,0.361200,0.361200,0.206550,0.206550,0.94510,0.196850,0.96940,0.969400,-0.090000,0.960600,0.524200,0.153475
1,0.153475,0.986100,0.9756,0.975500,0.975500,0.975500,0.97550,0.63690,0.636900,0.63690,0.63690,0.000000,0.630850,0.63085,0.956300,0.956300,0.9563,0.96620,0.296000,0.981400,0.000000,0.000000,0.296000,0.296000,0.15855,0.15855,0.97090,0.97090,0.97090,-0.29600,-0.29600,0.98580,0.76605,0.766050,0.610300,0.973800,0.803800,0.883650,0.970150,0.96710,...,0.973300,0.938933,0.890060,0.941325,0.871178,0.601850,0.660400,0.711050,0.115667,0.957500,0.969100,0.917567,0.656225,0.500080,0.943850,0.962800,0.491200,0.772450,0.574783,0.977850,0.000000,0.000000,0.970150,0.966100,0.966100,0.895050,0.951000,0.978150,0.782733,0.851620,0.860733,0.845600,0.91030,0.910300,0.73390,0.821631,0.549114,0.975300,0.879250,-0.449900
2,-0.449900,-0.772200,-0.7722,0.976400,0.984400,0.973600,0.97360,0.97360,0.271600,0.27160,0.27160,0.671933,0.671933,0.65670,0.932025,0.000000,0.0000,0.83245,-0.781800,-0.781800,-0.781800,-0.781800,0.733900,0.733900,0.00000,0.00000,0.49625,0.49625,0.49625,0.97425,0.97425,0.92750,0.92750,0.982600,0.982600,0.962150,0.962150,0.976650,0.976650,0.97470,...,0.886750,0.981600,0.607680,0.420200,0.951850,0.573200,0.798225,0.759600,0.039800,0.604200,0.826400,0.970900,0.307200,0.636900,0.636900,0.823350,0.977500,0.687400,0.976100,0.956400,0.156000,0.460567,0.460567,0.541525,0.710660,0.833067,0.349900,0.979800,0.810240,0.352333,0.499983,0.740200,-0.54230,0.973833,0.48780,0.652233,0.161250,0.161250,0.974900,0.176700
3,0.176700,-0.349850,0.9763,0.400200,0.072850,0.418400,0.41840,0.23055,0.976100,-0.22485,-0.17755,-0.177550,0.574467,0.29600,0.296000,0.296000,-0.0378,0.98000,0.980000,0.980000,0.102700,0.102700,0.102700,0.879300,0.87930,0.29600,-0.41520,-0.41520,-0.41520,0.98520,0.98520,0.98520,0.54070,0.984300,0.984300,0.984300,0.985300,0.985300,0.889300,-0.21440,...,0.713600,0.126180,0.953375,0.977050,0.978733,0.719425,0.719425,0.984300,0.085650,0.985000,0.985000,0.434800,0.667750,0.929100,0.929100,0.492200,0.981800,0.981800,0.981800,0.985700,0.985700,0.980550,0.040300,0.980800,0.980800,0.599960,0.970050,0.938414,0.859900,0.946080,0.977700,0.974425,0.65540,0.655400,0.70025,0.700250,0.700250,-0.624000,-0.624000,-0.624000
4,-0.624000,0.787600,0.7876,0.954900,-0.726900,0.923100,0.92310,0.70960,0.709600,0.34000,0.58590,0.585900,0.585900,0.98250,0.982500,0.982500,0.9825,0.98250,0.302000,0.302000,0.302000,0.302000,-0.289400,-0.289400,-0.28940,0.15780,0.15780,0.15780,0.29600,0.29600,0.84920,0.97470,0.97470,0.977600,0.977600,-0.128000,0.161100,0.161100,0.318200,0.31820,...,0.968500,0.980400,0.138467,0.978750,0.978750,0.978900,0.978900,0.419900,0.846633,0.544575,0.323900,-0.539100,0.838700,0.838700,0.753325,0.506750,0.735575,0.077080,0.315600,-0.097600,-0.097600,0.905100,0.835975,0.361200,0.708200,0.708200,0.454120,0.790700,0.894300,0.894300,0.977160,0.973167,0.84535,0.845350,0.24165,0.241650,0.742867,0.902450,0.000000,0.000000
5,0.000000,0.000000,0.1779,0.961700,0.961700,0.961700,0.96170,0.96170,0.000000,0.00000,0.00000,0.000000,0.000000,0.69610,0.696100,0.696100,0.9304,0.93040,0.93

Next get the prices for each day

In [130]:
pricesDf = pd.read_csv(ticker+'historical.csv')
pricesDf['Date'] = pd.to_datetime(pricesDf['Date'])
pricesDf.set_index('Date', inplace=True)
pricesDf = pricesDf.resample('D').pad()

In [121]:
outputdf['Close'] = np.nan
outputdf['PrevClose'] = np.nan
curdate = first_data_collection_date
for i in range(outputdf.shape[0]):
  outputdf.loc[i, 'PrevClose'] = pricesDf.loc[curdate.strftime("%Y-%m-%d")]['Close']
  curdate=first_data_collection_date+datetime.timedelta(days=i)
  try:
    outputdf.loc[i, 'Close'] = pricesDf.loc[curdate.strftime("%Y-%m-%d")]['Close']
  except:
    outputdf.loc[i, 'Close'] = 0 #Value not known yet
outputdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,Close,PrevClose
0,0.512100,0.375733,0.276267,0.345400,0.504680,0.324300,-0.078483,0.374300,-0.161500,0.089050,0.182275,0.310400,-0.012620,-0.012620,-0.708900,0.167700,0.310660,-0.234200,0.883400,0.000000,0.202150,0.202150,0.578140,0.399800,0.399800,0.060000,0.060000,0.163533,0.163533,0.247667,-0.216800,0.231350,0.231350,0.519300,-0.035300,0.550050,-0.172500,0.038100,0.669700,0.669700,...,0.084200,0.084683,0.418000,0.026540,0.041133,0.500960,0.252150,0.022600,-0.077700,0.377775,0.378950,0.080743,0.266200,0.011022,0.420687,0.250767,0.161150,0.247070,0.134340,-0.037129,-0.123167,-0.068500,0.004233,0.089071,0.101400,-0.082812,0.405550,0.384617,0.323025,-0.178945,-0.194527,0.231943,0.220200,0.099040,0.276789,0.509700,0.087640,0.037967,1114.000000,1114.000000
1,0.037967,0.378560,0.013743,0.266275,0.325633,0.034833,0.165500,0.165720,-0.136925,0.251800,-0.200950,-0.096233,0.111250,0.534600,0.211450,0.048750,-0.185800,0.052317,0.257267,0.659700,-0.447067,-0.031200,-0.031200,-0.458800,0.226133,0.275750,0.275750,0.280533,-0.024600,0.625100,-0.542300,-0.292225,-0.201580,0.481140,-0.065175,-0.010860,0.127780,0.106582,0.293531,0.239167,...,0.178462,0.087731,0.002893,0.273508,0.324366,0.271340,0.130072,0.084800,0.166168,0.247718,0.182276,0.235962,0.296864,0.218494,0.058772,0.218052,0.018200,0.156015,0.116673,-0.018444,0.143929,0.084752,0.079684,0.067860,0.087604,0.131343,0.192045,0.343405,0.089978,0.013350,-0.083031,0.196833,0.230603,0.117866,0.098285,0.177673,0.024524,0.100937,1208.589966,1114.000000
2,0.100937,0.085200,0.455285,0.223017,0.017036,0.270463,-0.073908,0.044208,0.172791,0.143823,0.298100,-0.015647,0.393522,0.251518,0.311953,0.090693,0.229860,0.502038,0.027975,0.100450,0.009725,0.250563,0.058662,0.295483,0.169195,0.103933,0.036978,0.122258,-0.085979,0.126100,-0.042087,0.177630,-0.069320,0.061654,0.210271,0.108367,0.044855,-0.030224,0.053919,-0.112730,...,0.281203,0.164485,0.474063,0.006863,0.059331,0.267130,0.162495,0.223339,0.097260,0.091174,0.115070,0.019633,0.255675,0.281190,0.161580,0.379820,0.229383,0.196473,0.054450,0.322800,0.179641,0.318169,0.178883,0.154162,0.269984,0.235205,0.296050,0.294455,-0.054000,0.296261,0.332760,0.649867,0.138427,0.359786,0.146927,0.214964,0.210827,0.006830,1172.000000,1208.589966
3,0.006830,0.013067,0.186286,0.173654,0.403637,0.063362,0.098000,0.464333,0.249364,0.226167,-0.017650,0.172167,0.115133,0.162464,0.100929,0.230644,0.172150,0.297250,-0.611550,0.044333,0.396467,0.497000,-0.233600,-0.080560,0.700083,0.420338,0.268258,0.229555,0.038292,0.206800,0.429175,0.331944,-0.038217,0.306829,0.144192,0.197588,0.200200,0.134950,0.258744,0.146467,...,0.195400,0.299747,0.136267,0.038081,0.226608,0.059365,0.001470,0.161238,0.101195,0.182450,0.085567,0.084273,0.149557,0.148073,0.191153,0.136506,0.307145,-0.020171,0.251231,0.299425,-0.281412,0.139125,0.036486,-0.014456,0.205357,0.168483,0.325830,0.369262,0.112547,0.021894,0.203392,0.242786,0.103136,0.124000,0.281062,0.248080,0.063700,-0.033217,1213.859985,1172.000000
4,-0.033217,0.076281,0.272812,0.340682,0.267791,-0.096743,0.280471,0.044371,-0.001085,0.296020,0.130700,0.223500,0.510533,0.151733,0.325125,0.449580,0.180600,-0.019050,0.027771,0.047020,0.208600,0.244933,0.361200,0.294280,0.162143,0.069117,-0.232725,0.037700,0.211950,-0.009510,0.107762,0.447300,-0.016750,0.166167,-0.058660,0.076957,0.300040,0.475608,0.384086,0.056569,...,0.206030,0.242864,0.208638,0.246529,0.118448,0.037762,0.271825,0.052165,0.036856,0.066735,0.109475,0.124117,0.238620,0.421629,0.079509,0.475970,0.185857,0.051394,0.098100,0.138119,0.076170,0.041433,0.036343,0.177660,0.175000,0.263747,0.447313,0.098479,0.106225,-0.098322,0.096079,0.063005,0.240433,-0.050490,0.235250,-0.177814,0.048994,0.025600,1229.910034,1213.8599

Basic neural network

In [88]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import regularizers

In [89]:
target = outputdf.pop('Close')

In [90]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(outputdf)

In [91]:
def get_basic_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.l2(1e-4), activity_regularizer=regularizers.l2(1e-5)), 
    tf.keras.layers.Dense(8, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.l2(1e-4), activity_regularizer=regularizers.l2(1e-5)),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.MeanAbsolutePercentageError(reduction="auto", name="MeanAbsolutePercentageError"))
  return model

In [48]:
model = get_basic_model()
#Model to predict future value
model.fit(outputdf[:-1], target[:-1], epochs=500, batch_size=5)

#If we have validation data
#model.fit(outputdf[:-1], target[:-1], epochs=500, batch_size=5, validation_data=[outputdf[-1:], target[-1:]])

Epoch 1/500
4/4 [==============================] - 1s 4ms/step - loss: 95.9658
Epoch 2/500
4/4 [==============================] - 0s 4ms/step - loss: 94.8524
Epoch 3/500
4/4 [==============================] - 0s 4ms/step - loss: 93.7963
Epoch 4/500
4/4 [==============================] - 0s 4ms/step - loss: 92.7285
Epoch 5/500
4/4 [==============================] - 0s 4ms/step - loss: 91.6377
Epoch 6/500
4/4 [==============================] - 0s 6ms/step - loss: 90.5201
Epoch 7/500
4/4 [==============================] - 0s 4ms/step - loss: 89.3611
Epoch 8/500
4/4 [==============================] - 0s 4ms/step - loss: 88.1694
Epoch 9/500
4/4 [==============================] - 0s 4ms/step - loss: 86.9247
Epoch 10/500
4/4 [==============================] - 0s 6ms/step - loss: 85.6322
Epoch 11/500
4/4 [==============================] - 0s 4ms/step - loss: 84.3061
Epoch 12/500
4/4 [==============================] - 0s 5ms/step - loss: 82.9019
Epoch 13/500
4/4 [==============================]

In [49]:
print("Predicted value: %f"%(model.predict(outputdf)[-1:][0][0]))

#For use with validation 
#print("Predicted value: %f, True value: %f"%(model.predict(outputdf)[-1:][0][0], target[-1:].values[0]))
#print("Percent error on unseen datapoint: %f"%(((target[-1:].values[0] - model.predict(outputdf)[-1:][0][0]) / target[-1:].values[0])*100))

Predicted value: 35.974365
